# Making RNN learn addition, subtraction, multiplication and division

In [1]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

from termcolor import colored

## Generating Data

In [2]:
all_chars = "1234567890+-/*"
num_features = len(all_chars)
num_features

14

In [3]:
# Tokenizing the characters
char_to_idx = {c:i for i,c in enumerate(all_chars)}
idx_to_char = {i:c for i,c in enumerate(all_chars)}

In [38]:
def random_operation(num1, num2):
    """Applies random operation from `+ - * /` and return result and operator"""
    
    operator = np.random.choice(['+', '-', '*', '/'])
    result = 0
    if operator == '+':
        result = num1 + num2
    elif operator == '-':
        result = num1 - num2
    elif operator == '*':
        result = num1 * num2
    elif operator == '/':
        result = num1 / num2

    return result, operator

In [77]:
random_operation(5,5)

(1.0, '/')

In [94]:
def generate_data():
    """Generates sample(str) - results(str) pair """
    first_num = np.random.randint(0,100)
    second_num = np.random.randint(0,100)
    res, opr = random_operation(first_num, second_num)
    sample = str(first_num) + opr + str(second_num)
    label = str(res)
    return (sample, label)

# Test
generate_data()

('64-31', '33')

## Create the Model

In [ ]:
hidden_units = 128
max_timesteps = 5

model = Sequential([
    # Encoder
    SimpleRNN(hidden_units, input_shape=(None, num_features)),
    RepeatVector(max_timesteps),
    # Decoder
    SimpleRNN(hidden_units, return_sequences=True),
    TimeDistributed(Dense(num_features, activation="softmax"))
])

model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

## Vectorize and De-Vectorize Data

In [ ]:
 def vectorize_sample(sample, label):
    x = np.zeros((max_timesteps, num_features))
    y = np.zeros((max_timesteps, num_features))
    
    diff_x = max_timesteps - len(sample)
    diff_y = max_timesteps - len(label)
    
    for i, c in enumerate(sample):
        x[diff_x + i, char_to_idx[c]] = 1
    for i in range(diff_x):
        x[i, char_to_idx['0']] = 1
    
    for i, c in enumerate(label):
        y[diff_y + i, char_to_idx[c]] = 1
    for i in range(diff_y):
        y[i, char_to_idx['0']] = 1
    
    return (x,y)

In [ ]:
s1,l1 = generate_data()
print(s1 +' = ' + l1)
s1v, l1v = vectorize_sample(s1, l1)
# print(s1v)
# print(l1v)

In [8]:
def devectorize_sample(sample):
    sample_char_list = [idx_to_char[i] for i in np.argmax(sample, axis=1)]
    sample = ''.join(sample_char_list).lstrip('0')
    return sample

In [9]:
devectorize_sample(l1v)

'86'

## Creating Trainset

In [10]:
def create_dataset(num_sample=5000):
    x = np.zeros((num_sample, max_timesteps, num_features))
    y = np.zeros((num_sample, max_timesteps, num_features))
    
    for i in range(num_sample):
        s, l = generate_data()
        sv, lv = vectorize_sample(s,l)
        x[i] = sv
        y[i] = lv
    
    return x, y

In [11]:
x_train, y_train = create_dataset()
print(x_data.shape, y_data.shape)

(5000, 5, 11) (5000, 5, 11)


In [12]:
devectorize_sample(x_train[0]), devectorize_sample(y_train[0])

('69+27', '96')

## Training the Model

In [ ]:
lb_cb = LambdaCallback(
    on_epoch_end=lambda e, l: print('->{:.2f}'.format(l['val_acc']), end='')
)
es_cb = EarlyStopping(monitor='val_loss', patience=5)

model.fit(x_train, y_train, epochs=100, batch_size=256, validation_split=0.1,
          callbacks=[lb_cb, es_cb], verbose=0)

In [14]:
x_test, y_test = create_dataset(10)
y_pred = model.predict(x_test)

for i, pred in enumerate(y_pred):
    y = devectorize_sample(y_test[i])
    y_hat = devectorize_sample(pred)
    col = 'green'
    if y!=y_hat:
        col = 'red'
    out = 'Sample: ' + devectorize_sample(x_test[i]) + ' Actual: ' + y + ' Predicted: ' + y_hat
    print(colored(out, col))

Sample: 61+91 Actual: 152 Predicted: 152
Sample: 93+21 Actual: 114 Predicted: 114
Sample: 90+39 Actual: 129 Predicted: 129
Sample: 87+30 Actual: 117 Predicted: 117
Sample: 85+51 Actual: 136 Predicted: 136
Sample: 73+84 Actual: 157 Predicted: 157
Sample: 34+46 Actual: 80 Predicted: 80
Sample: 50+91 Actual: 141 Predicted: 141
Sample: 2+15 Actual: 17 Predicted: 17
Sample: 39+80 Actual: 119 Predicted: 119


In [15]:
x_test, y_test = create_dataset(1000)
_, acc = model.evaluate(x_test, y_test, verbose=0)
print('Test Accuracy ::', acc)

Test Accuracy :: 0.9931999931335449
